In [20]:
!pip install selenium

In [21]:
!pip install webdriver-manager

In [22]:
!pip install openpyxl

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import tqdm as tqdm
from tqdm import tqdm 
import pandas as pd

In [24]:
# Path ke chromedriver
chrome_driver_path = r"C:\Windows\chromedriver.exe"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

url = "https://sinta.kemdiktisaintek.go.id/subjects/detail/52338"
driver.get(url)

In [25]:
TARGET = 3000
time.sleep(3)

all_links = set()

pbar = tqdm(total=TARGET, desc="Mengumpulkan author links")

while True:
    author_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/authors/profile/']")
    for link in author_links:
        href = link.get_attribute("href")
        if href and href not in all_links:
            all_links.add(href)
            pbar.update(1) 

    if len(all_links) >= TARGET:
        print("Sudah terkumpul 5000 link author, berhenti.")
        break

    try:
        next_button = driver.find_element(By.LINK_TEXT, "Next")
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)
    except:
        print("Tidak ada tombol Next lagi, selesai.")
        break

pbar.close()

for i, l in enumerate(list(all_links)[:TARGET], 1):
    print(i, l)

# driver.quit()

Mengumpulkan author links: 100%|█████████▉| 2995/3000 [1:45:49<00:10,  2.12s/it] 

Tidak ada tombol Next lagi, selesai.
1 https://sinta.kemdiktisaintek.go.id/authors/profile/129111
2 https://sinta.kemdiktisaintek.go.id/authors/profile/5985185
3 https://sinta.kemdiktisaintek.go.id/authors/profile/6115448
4 https://sinta.kemdiktisaintek.go.id/authors/profile/5997122
5 https://sinta.kemdiktisaintek.go.id/authors/profile/5998758
6 https://sinta.kemdiktisaintek.go.id/authors/profile/6734528
7 https://sinta.kemdiktisaintek.go.id/authors/profile/6748562
8 https://sinta.kemdiktisaintek.go.id/authors/profile/3494
9 https://sinta.kemdiktisaintek.go.id/authors/profile/5976224
10 https://sinta.kemdiktisaintek.go.id/authors/profile/6730487
11 https://sinta.kemdiktisaintek.go.id/authors/profile/6095138
12 https://sinta.kemdiktisaintek.go.id/authors/profile/6023445
13 https://sinta.kemdiktisaintek.go.id/authors/profile/6001360
14 https://sinta.kemdiktisaintek.go.id/authors/profile/6155609
15 https://sinta.kemdiktisaintek.go.id/authors/profile/6755583
16 https://sinta.kemdiktisainte

In [26]:
links = list(all_links)

In [27]:
links

['https://sinta.kemdiktisaintek.go.id/authors/profile/129111',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/5985185',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6115448',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/5997122',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/5998758',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6734528',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6748562',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/3494',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/5976224',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6730487',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6095138',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6023445',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6001360',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6155609',
 'https://sinta.kemdiktisaintek.go.id/authors/profile/6755583',
 'https://sinta.kemdiktisaintek.go.id/author

In [28]:
from IPython.display import FileLink

with open("links.txt", "w") as f:
    for item in links:
        f.write(str(item) + "\n")

FileLink("links.txt")

e:\Python Workspace Aur\links.txt

In [29]:
def quick_text(elem):
    return elem.text.strip() if elem else ""

data = []

for i, url in enumerate(tqdm(links, desc="Scraping progress")):
    driver.get(url)
    time.sleep(1.5)  # kasih jeda supaya elemen muncul

    # Nama
    try:
        nama_elem = driver.find_element(By.CSS_SELECTOR, "h3 a")
        nama = quick_text(nama_elem)
    except:
        nama = ""

    # Perguruan Tinggi & Prodi
    try:
        meta_elem = driver.find_element(By.CSS_SELECTOR, ".meta-profile")
        meta_text = meta_elem.text.strip().split("\n")
        perguruan_tinggi = meta_text[0] if len(meta_text) > 0 else ""
        program_studi    = meta_text[1] if len(meta_text) > 1 else ""
    except:
        perguruan_tinggi = ""
        program_studi = ""

    # Bidang
    bidang_elems = driver.find_elements(By.CSS_SELECTOR, ".profile-subject a")
    bidang = ", ".join([quick_text(b) for b in bidang_elems])

    # Sinta Score
    skor_elems = driver.find_elements(By.CSS_SELECTOR, ".stat-profile .pr-num")
    skor = [quick_text(s) for s in skor_elems]
    sinta_overall, sinta_3yr, affil_overall, affil_3yr = (skor + ["", "", "", ""])[:4]

    data.append({
        "url": url,
        "nama": nama,
        "perguruan_tinggi": perguruan_tinggi,
        "program_studi": program_studi,
        "bidang": bidang,
        "sinta_score_overall": sinta_overall,
        "sinta_score_3yr": sinta_3yr,
        "affil_score_overall": affil_overall,
        "affil_score_3yr": affil_3yr
    })

    if i < 3:  # preview 3 data pertama
        print(f"\nData ke-{i+1}: {data[-1]}")

    time.sleep(0.3)


Scraping progress:   0%|          | 0/2995 [00:00<?, ?it/s]


Data ke-1: {'url': 'https://sinta.kemdiktisaintek.go.id/authors/profile/129111', 'nama': 'SULISWORO KUSDIYATI', 'perguruan_tinggi': 'Universitas Islam Bandung', 'program_studi': 'Unknown', 'bidang': 'Psikologi, Psikologi Perkembangan, Psikologi Pendidikan, Psikologi Klinis Anak, Kecerdasan', 'sinta_score_overall': '550', 'sinta_score_3yr': '224', 'affil_score_overall': '559', 'affil_score_3yr': '224'}


Scraping progress:   0%|          | 1/2995 [00:02<2:04:29,  2.49s/it]


Data ke-2: {'url': 'https://sinta.kemdiktisaintek.go.id/authors/profile/5985185', 'nama': 'RAKHMAT PURNOMO', 'perguruan_tinggi': 'Universitas Bhayangkara Jakarta Raya', 'program_studi': 'S1 - Informatika', 'bidang': 'Artificial Intelligence, Data science, Software Engineering, Computer netowork', 'sinta_score_overall': '458', 'sinta_score_3yr': '252', 'affil_score_overall': '458', 'affil_score_3yr': '252'}


Scraping progress:   0%|          | 2/2995 [00:04<1:57:33,  2.36s/it]


Data ke-3: {'url': 'https://sinta.kemdiktisaintek.go.id/authors/profile/6115448', 'nama': 'MERI SRI WAHYUNI', 'perguruan_tinggi': 'Sekolah Tinggi Manajemen Informatika dan Komputer Triguna Dharma', 'program_studi': 'S1 - Sistem Informasi', 'bidang': 'data mining, expert system, embed system', 'sinta_score_overall': '161', 'sinta_score_3yr': '96', 'affil_score_overall': '0', 'affil_score_3yr': '0'}


Scraping progress: 100%|██████████| 2995/2995 [2:11:31<00:00,  2.63s/it]  


In [30]:
df = pd.DataFrame(data)

df.to_csv("sinta_authors_revisi2.csv", index=False, encoding="utf-8-sig")
df.to_excel("sinta_authors_revisi2.xlsx", index=False)
print("Data berhasil disimpan ke 'sinta_authors_revisi2.csv' dan 'sinta_authors_revisi2.xlsx'")

Data berhasil disimpan ke 'sinta_authors_revisi2.csv' dan 'sinta_authors_revisi2.xlsx'


In [31]:
df

,url,nama,perguruan_tinggi,program_studi,bidang,sinta_score_overall,sinta_score_3yr,affil_score_overall,affil_score_3yr
0,https://sinta.kemdiktisaintek.go.id/authors/pr...,SULISWORO KUSDIYATI,Universitas Islam Bandung,Unknown,"Psikologi, Psikologi Perkembangan, Psikologi P...",550,224,559,224
1,https://sinta.kemdiktisaintek.go.id/authors/pr...,RAKHMAT PURNOMO,Universitas Bhayangkara Jakarta Raya,S1 - Informatika,"Artificial Intelligence, Data science, Softwar...",458,252,458,252
2,https://sinta.kemdiktisaintek.go.id/authors/pr...,MERI SRI WAHYUNI,Sekolah Tinggi Manajemen Informatika dan Kompu...,S1 - Sistem Informasi,"data mining, expert system, embed system",161,96,0,0
3,https://sinta.kemdiktisaintek.go.id/authors/pr...,AMIR MURTAKO,Universitas Pancasila,S1 - Teknik Informatika,Artificial Intelligence,204,53,0,0
4,https://sinta.kemdiktisaintek.go.id/authors/pr...,RAHMAD AL RIAN,Universitas Muhammadiyah Riau,S1 - Pendidikan Informatika,"Information Technology, Data Analyst, Big Data...",45,4,0,0
...,...,...,...,...,...,...,...,...,...
2990,https://sinta.kemdiktisaintek.go.id/authors/pr...,I KETUT PURNAMAWAN,Universitas Pendidikan Ganesha,S1 - Ilmu Komputer,Artificial Intelligence,170,21,0,0
2991,https://sinta.kemdiktisaintek.go.id/authors/pr...,GUNAWAN RAHARJO,Sekolah Tinggi Keguruan dan Ilmu Pendidikan Na...,Unknown,"Artificial Intelligence, Machine Learning, Dat...",1,1,200,200
2992,https://sinta.kemdiktisaintek.go.id/authors/pr...,YAQUTINA MARJANI SANTOSA,Politeknik Negeri Indramayu,D4 - Rekayasa Perangkat Lunak,Kecerdasan Buatan,169,149,169,149
2993,https://sinta.kemdiktisaintek.go.id/authors/pr...,NURRAUDYA TUZ ZAHRA,Politeknik Negeri Padang,D4 - Teknologi Rekayasa Perangkat Lunak,"Artificial Intelligence, Computer Vision, Inst...",2,1,2,1


In [32]:
df.isnull().sum()

url                    0
nama                   0
perguruan_tinggi       0
program_studi          0
bidang                 0
sinta_score_overall    0
sinta_score_3yr        0
affil_score_overall    0
affil_score_3yr        0
dtype: int64

In [33]:
driver.quit()